# Climate Change IFD Projection

Notebook to calculate new IFD tables based on the climate change loadings given in The Australian Rainfall and Runoff: A Guide to Flood Estimation, Book 1, chapter 6.

In [554]:
import pandas as pd
import numpy as np

In [555]:
ifd_path = './depths_upper_all_design.csv'  # the path to the benchmark BOM IFD data.
climate_year = 2024                         # the year of the climate scenario.
ssp = 'SSP2'                                # Shared Socioeconomic Pathways 2.
ifd = pd.read_csv(ifd_path, header=7)       # read the benchmark BOM IFD data.
duration = ifd['Duration']                  # extract the duration column.
ifd.drop('Duration', axis=1, inplace=True)  # drop the duration column.

In [556]:
def alpha(dur):
    if dur <= 60:
        return 0.15
    elif dur >= 1440:
        return 0.08
    else:
        x = [60, 90, 120, 180, 270, 360, 540, 720, 1080, 1440]
        y = [0.15, 0.137, 0.128, 0.118, 0.108, 0.102, 0.095, 0.090, 0.084, 0.08]
        deg = 6
        coeff = np.polyfit(x, y, deg)
        return sum([coeff[-(i+1)] * dur ** i for i in range(deg, -1, -1)])

def ifd_coeffecient(dT, alpha):
    # Project the IFD data to the climate year.
    return (1 + alpha) ** dT

def delta_temp(scenario, dt):
    data = np.array([[1.2, 1.2, 1.2, 1.3],
                    [1.4, 1.7, 1.8, 2.1],
                    [1.5, 2.4, 3.3, 4.1]])
    
    if scenario == 'SSP1':
        x = 0
    elif scenario == 'SSP2':
        x = 1
    elif scenario == 'SSP3':
        x = 2
    elif scenario == 'SSP5':
        x = 3
    else:
        raise ValueError('Invalid scenario.')
    
    if dt <= 2040:
        y = 0
    elif dt <= 2060:
        y = 1
    elif dt < 2080:
        raise ValueError('No data for this year.')
    elif dt <= 2100:
        y = 2
    else:
        raise ValueError('No data for this year.')
    return data[y, x]

In [557]:
ifd

,Duration in min,12EY,6EY,4EY,3EY,2EY,63.2%,50%,0.5EY,20%,0.2EY,10%,5%,2%,1%,1 in 200,1 in 500,1 in 1000,1 in 2000
0,1.0,0.479,0.535,0.639,0.717,0.837,1.07,1.21,1.35,1.69,1.73,2.05,2.42,2.95,3.38,3.86,4.47,4.96,5.48
1,2.0,0.758,0.859,1.050,1.190,1.410,1.84,2.07,2.30,2.81,2.86,3.31,3.82,4.45,4.93,5.57,6.45,7.16,7.90
2,3.0,0.985,1.120,1.370,1.560,1.860,2.45,2.76,3.06,3.76,3.84,4.47,5.18,6.09,6.81,7.71,8.93,9.91,10.90
3,4.0,1.180,1.340,1.630,1.870,2.230,2.94,3.33,3.69,4.57,4.67,5.47,6.38,7.60,8.57,9.74,11.30,12.50,13.80
4,5.0,1.350,1.530,1.870,2.130,2.540,3.36,3.80,4.22,5.27,5.38,6.33,7.42,8.93,10.20,11.60,13.40,14.90,16.40
5,10.0,2.030,2.280,2.760,3.130,3.710,4.85,5.51,6.12,7.75,7.90,9.42,11.20,13.70,15.90,18.20,21.00,23.40,25.80
6,15.0,2.540,2.850,3.420,3.870,4.550,5.88,6.68,7.42,9.41,9.59,11.40,13.60,16.80,19.40,22.20,25.70,28.60,31.50
7,20.0,2.960,3.310,3.970,4.470,5.240,6.71,7.62,8.45,10.70,10.90,13.00,15.40,19.00,21.90,25.00,29.00,32.20,35.50
8,25.0,3.320,3.720,4.450,5.000,5.830,7.42,8.42,9.34,11.80,12.00,14.30,16.90,20.70,23.80,27.20,31.50,35.00,38.60
9,30.0,3.650,4.080,4.880,5.470,6.370,8.06,9.13,10.10,12.70,13.00,15.40,18.20,22.20,25.40,29.00,33.60,37.30,41.10


In [558]:
a = np.array([alpha(i) for i in ifd['Duration in min']])
x = np.array([ifd_coeffecient(delta_temp(ssp, climate_year), i) for i in a])
pd.DataFrame(zip(ifd['Duration in min'], x), columns=['Durations', 'IFD Coeffecient'])

,Durations,IFD Coeffecient
0,1.0,1.182599
1,2.0,1.182599
2,3.0,1.182599
3,4.0,1.182599
4,5.0,1.182599
5,10.0,1.182599
6,15.0,1.182599
7,20.0,1.182599
8,25.0,1.182599
9,30.0,1.182599


In [559]:
projected_ifd = ifd.iloc[:, 1:].multiply(x, axis=0)
projected_ifd.insert(0, 'Duration in min', ifd['Duration in min'])
projected_ifd

,Duration in min,12EY,6EY,4EY,3EY,2EY,63.2%,50%,0.5EY,20%,0.2EY,10%,5%,2%,1%,1 in 200,1 in 500,1 in 1000,1 in 2000
0,1.0,0.566465,0.632690,0.755681,0.847923,0.989835,1.265381,1.430944,1.596508,1.998592,2.045896,2.424327,2.861889,3.488666,3.997184,4.564831,5.286216,5.865690,6.480641
1,2.0,0.896410,1.015852,1.241729,1.407292,1.667464,2.175982,2.447979,2.719977,3.323102,3.382232,3.914402,4.517527,5.262564,5.830212,6.587075,7.627762,8.467407,9.342530
2,3.0,1.164860,1.324511,1.620160,1.844854,2.199634,2.897367,3.263972,3.618752,4.446571,4.541179,5.286216,6.125861,7.202026,8.053497,9.117836,10.560607,11.719553,12.890326
3,4.0,1.395467,1.584682,1.927636,2.211460,2.637195,3.476840,3.938054,4.363789,5.404476,5.522736,6.468815,7.544980,8.987750,10.134871,11.518512,13.363366,14.782484,16.319862
4,5.0,1.596508,1.809376,2.211460,2.518935,3.003801,3.973532,4.493875,4.990567,6.232295,6.362381,7.485850,8.774883,10.560607,12.062507,13.718145,15.846823,17.620721,19.394619
5,10.0,2.400675,2.696325,3.263972,3.701534,4.387441,5.735604,6.516119,7.237504,9.165140,9.342530,11.140080,13.245106,16.201603,18.803320,21.523297,24.834573,27.672810,30.511047
6,15.0,3.003801,3.370406,4.044488,4.576657,5.380824,6.953681,7.899760,8.774883,11.128254,11.341122,13.481626,16.083343,19.867659,22.942415,26.253692,30.392787,33.822324,37.251860
7,20.0,3.500492,3.914402,4.694917,5.286216,6.196817,7.935237,9.011402,9.992959,12.653806,12.890326,15.373783,18.212020,22.469376,25.898912,29.564968,34.295363,38.079679,41.982255
8,25.0,3.926228,4.399267,5.262564,5.912994,6.894551,8.774883,9.957481,11.045472,13.954665,14.191185,16.911162,19.985919,24.479794,28.145850,32.166685,37.251860,41.390956,45.648311
9,30.0,4.316485,4.825003,5.771082,6.468815,7.533154,9.531746,10.797126,11.944247,15.019004,15.373783,18.212020,21.523297,26.253692,30.038008,34.295363,39.735317,44.110933,48.604808


In [560]:
with open(ifd_path, 'r') as f:
    lines = f.readlines()
header = lines[:9]

with open(f'depths_upper_all_design_{climate_year}.csv', 'w', newline='') as f:
    f.writelines(header)
    projected_ifd.insert(0, 'Duration', duration)
    projected_ifd.to_csv(f, index=False, float_format='%.3g')